In [1]:
!pip install -U langchain langchain-community langchain-huggingface faiss-cpu sentence-transformers pypdf huggingface_hub

   ---------------------------------------- 0.0/1.0 MB ? eta -:--:--
   ---------- ----------------------------- 0.3/1.0 MB ? eta -:--:--
   -------------------- ------------------- 0.5/1.0 MB 1.7 MB/s eta 0:00:01
   -------------------- ------------------- 0.5/1.0 MB 1.7 MB/s eta 0:00:01
   -------------------- ------------------- 0.5/1.0 MB 1.7 MB/s eta 0:00:01
   ------------------------------ --------- 0.8/1.0 MB 799.2 kB/s eta 0:00:01
   ------------------------------ --------- 0.8/1.0 MB 799.2 kB/s eta 0:00:01
   ---------------------------------------- 1.0/1.0 MB 645.1 kB/s eta 0:00:00
   ---------------------------------------- 0.0/2.5 MB ? eta -:--:--
   ---- ----------------------------------- 0.3/2.5 MB ? eta -:--:--
   ------------ --------------------------- 0.8/2.5 MB 2.4 MB/s eta 0:00:01
   -------------------- ------------------- 1.3/2.5 MB 2.3 MB/s eta 0:00:01
   ----------------------------- ---------- 1.8/2.5 MB 2.2 MB/s eta 0:00:01
   -------------------------------

In [22]:
#Setup and API Key Configuration
import os
from getpass import getpass

def setup_huggingface_api():
    print("Setting up HuggingFace API Token")
    print("=" * 50)

    # Check if token already exists in environment
    api_token = os.environ.get('HUGGINGFACEHUB_API_TOKEN')

    if api_token:
        print("API token found in environment variables")
        return api_token
    else:
        print("Please enter your HuggingFace API token")
        print("Get your token from: https://huggingface.co/settings/tokens")
        api_token = getpass("Enter your HF API token: ")

        os.environ['HUGGINGFACEHUB_API_TOKEN'] = api_token
        print("API token configured successfully!")
        return api_token

api_token = setup_huggingface_api()


Setting up HuggingFace API Token
Please enter your HuggingFace API token
Get your token from: https://huggingface.co/settings/tokens
API token configured successfully!


In [25]:
# PDF to Vectors - Convert PDF into searchable database
import os
import pickle
from langchain.document_loaders import PyPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_huggingface import HuggingFaceEmbeddings
from langchain.vectorstores import FAISS

def pdf_to_vectors_simple(pdf_path):
    print("Converting PDF to Vector Database")
    print("=" * 50)

    print(f"Loading PDF: {pdf_path}")
    if not os.path.exists(pdf_path):
        print(f"File not found: {pdf_path}")
        return False


    try:
        loader = PyPDFLoader(pdf_path)
        documents = loader.load()
        print(f"Loaded {len(documents)} pages")
    except Exception as e:
        print(f"Error loading PDF: {e}")
        return False


    print("Splitting document into smaller chunks...")
    text_splitter = RecursiveCharacterTextSplitter(
        chunk_size=1000,        
        chunk_overlap=200,     
        length_function=len,
    )


    texts = text_splitter.split_documents(documents)
    print(f"✅ Created {len(texts)} text chunks")


    print("Creating embeddings (this converts text to numbers)...")
    embeddings = HuggingFaceEmbeddings(
        model_name="sentence-transformers/all-MiniLM-L6-v2"
    )

  
    print("Creating vector database...")
    vectorstore = FAISS.from_documents(texts, embeddings)

    
    vectorstore.save_local("faiss_index")
    print("Vector database saved as 'faiss_index'")

    
    metadata = {
        'total_chunks': len(texts),
        'total_pages': len(documents),
        'chunk_size': 1000,
        'overlap': 200,
        'pdf_name': pdf_path
    }

    with open("pdf_metadata.pkl", "wb") as f:
        pickle.dump(metadata, f)
    print("Metadata saved")

    print("\nPDF conversion complete!")
    print(f"Summary:")
    print(f"   -Pages processed: {len(documents)}")
    print(f"   -Chunks created: {len(texts)}")
    print(f"   -Ready for questions!")

    return True


pdf_to_vectors_simple("Simple Guide to Engineering College Counselling in-1.pdf")


Converting PDF to Vector Database
Loading PDF: Simple Guide to Engineering College Counselling in-1.pdf
Loaded 2 pages
Splitting document into smaller chunks...
✅ Created 4 text chunks
Creating embeddings (this converts text to numbers)...
Creating vector database...
Vector database saved as 'faiss_index'
Metadata saved

PDF conversion complete!
Summary:
   -Pages processed: 2
   -Chunks created: 4
   -Ready for questions!


True

In [ ]:
#Question Answering System with memory
import os
import pickle
from langchain.vectorstores import FAISS
from langchain_huggingface import HuggingFaceEmbeddings
from langchain.memory import ConversationBufferMemory
from huggingface_hub import InferenceClient
from langchain.memory import ChatMessageHistory
from langchain.schema import BaseMessage

def initialize_qa_system():
    print("Initializing Question-Answering System with Memory")
    print("=" * 50)


    if not os.path.exists("faiss_index"):
        print("Vector database not found!")
        print("Please run 'pdf_to_vectors_simple()' first")
        return None, None, None

    
    print("Loading vector database...")
    embeddings = HuggingFaceEmbeddings(
        model_name="sentence-transformers/all-MiniLM-L6-v2"
    )
    vectorstore = FAISS.load_local("faiss_index", embeddings,
                                 allow_dangerous_deserialization=True)

    
    
    print("Setting up conversation memory...")

    chat_history = ChatMessageHistory()
    memory = ConversationBufferMemory(
    chat_memory=chat_history,
    memory_key="chat_history",
    return_messages=True
    )

    
    print("Connecting to HuggingFace...")
    try:
        api_token = os.environ.get('HUGGINGFACEHUB_API_TOKEN')
        client = InferenceClient(api_key=api_token)
        print("HuggingFace client ready!")
    except Exception as e:
        print(f"HuggingFace connection failed: {e}")
        client = None

    return vectorstore, client, memory


In [ ]:
def ask_question_with_memory(question, vectorstore, client, memory, num_results=5):
    print(f"\nSearching for: '{question}'")

    
    docs_with_scores = vectorstore.similarity_search_with_score(question, k=num_results)

    print(f"Found {len(docs_with_scores)} relevant sections:")

    
    context_parts = []
    for i, (doc, score) in enumerate(docs_with_scores):
        page = doc.metadata.get('page', 'Unknown')
        print(f"   • Section {i+1}: Page {page} (Relevance: {score:.3f})")
        context_parts.append(f"[Page {page}]: {doc.page_content}")

    context = "\n\n".join(context_parts)
    

    
    chat_history = memory.chat_memory.messages if memory.chat_memory.messages else []
    history_text = "\n".join([f"{msg.type}: {msg.content}" for msg in chat_history[-4:]])

   
    print("Generating answer with conversation context...")

    if client:
        try:
            messages = [
                {
                    "role": "system",
                    "content": f"You are a helpful assistant. Answer questions based on the provided document context and conversation history. Always mention page numbers when possible.\n\nConversation History:\n{history_text}"
                },
                {
                    "role": "user",
                    "content": f"Document Context:\n{context}\n\nCurrent Question: {question}\n\nPlease provide a clear answer considering both the document and our conversation:"
                }
            ]

            response = client.chat_completion(
                messages=messages,
                model="meta-llama/Meta-Llama-3-8B-Instruct",
                max_tokens=300
            )

            answer = response.choices[0].message.content

            
            memory.chat_memory.add_user_message(question)
            memory.chat_memory.add_ai_message(answer)

            return answer

        except Exception as e:
            print(f"AI generation failed: {e}")
            print("Providing context-based answer instead...")

    
    answer = f"Based on your question and our conversation, here are the most relevant sections:\n\n{context[:1500]}..."
    memory.chat_memory.add_user_message(question)
    memory.chat_memory.add_ai_message(answer)

    return answer

In [29]:
def qa_chat():
    print("PDF Q&A Chat with Memory!")
    print("=" * 50)

    
    vectorstore, client, memory = initialize_qa_system()
    if vectorstore is None:
        return

    
    try:
        with open("pdf_metadata.pkl", "rb") as f:
            metadata = pickle.load(f)
        print(f"Loaded: {metadata['total_chunks']} chunks from {metadata['total_pages']} pages")
    except:
        print("Metadata not available")

    print("\nInstructions:")
    print("   - Ask questions - I'll remember our conversation!")
    print("   - Try follow-up questions like 'Can you explain that more?'")
    print("   - Type 'memory' to see conversation history")
    print("   - Type 'clear' to clear memory")
    print("   - Type 'quit' to exit")
    print("=" * 50)

    while True:
        question = input("\n Your question: ").strip()

        if question.lower() in ['quit', 'exit', 'bye']:
            print("Goodbye!")
            break

        if question.lower() == 'memory':
            messages = memory.chat_memory.messages
            if messages:
                print("Conversation History:")
                for i, msg in enumerate(messages[-6:]):  
                    role = "You" if msg.type == "human" else "Bot"
                    print(f"   {role}: {msg.content[:100]}...")
            else:
                print("No conversation history yet")
            continue

        if question.lower() == 'clear':
            memory.clear()
            print("Memory cleared!")
            continue

        if question.lower() == 'info':
            try:
                with open("pdf_metadata.pkl", "rb") as f:
                    metadata = pickle.load(f)
                print(f"System Status:")
                print(f"   - PDF: {metadata['pdf_name']}")
                print(f"   - Pages: {metadata['total_pages']}")
                print(f"   - Chunks: {metadata['total_chunks']}")
                print(f"   - AI Client: {'Active' if client else 'Using fallback'}")
                print(f"   - Memory: {len(memory.chat_memory.messages)} messages stored")
            except:
                print("System info not available")
            continue

        if not question:
            print("Please enter a question!")
            continue

        
        answer = ask_question_with_memory(question, vectorstore, client, memory)

        print(f"\n**Answer:**")
        print(answer)
        print("-" * 70)


qa_chat()


PDF Q&A Chat with Memory!
Initializing Question-Answering System with Memory
📂 Loading vector database...
Setting up conversation memory...
Connecting to HuggingFace...
HuggingFace client ready!
Loaded: 4 chunks from 2 pages

Instructions:
   - Ask questions - I'll remember our conversation!
   - Try follow-up questions like 'Can you explain that more?'
   - Type 'memory' to see conversation history
   - Type 'clear' to clear memory
   - Type 'quit' to exit

🔍 Searching for: 'what is this pdf about?'
📄 Found 4 relevant sections:
   • Section 1: Page 0 (Relevance: 1.888)
   • Section 2: Page 0 (Relevance: 1.908)
   • Section 3: Page 1 (Relevance: 2.010)
   • Section 4: Page 1 (Relevance: 2.061)
🤖 Generating answer with conversation context...

**Answer:**
This PDF is about the college counseling process in Tamil Nadu, specifically for engineering college admissions. It provides a simple guide to understanding the process, explaining how marks from Class 12 exams determine college choice